In [112]:
!pip install pytorch_tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [128]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier


from google.colab import files
uploaded = files.upload()


Saving train.csv to train (6).csv
Saving test.csv to test (8).csv
Saving sample_submission.csv to sample_submission (6).csv


In [146]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

x_train = train
y_train = train['Y_Class']


drop_x = []
for i in range(1, 2876):
  tmp = 'X_'+str(i)
  if len(x_train[tmp].unique()) > 10: #기준을 더 낮추어봄 // 16 - 0.62 , 10 - 0.7177629134, 6 - 0.5987883711
    pass
  else:
    drop_x.append(tmp)

#test set 사용
test_x = test.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])
x_train = x_train.drop(columns = ['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
x_train = x_train.drop(columns = drop_x)
test_x = test_x.drop(columns = drop_x)
x_train = x_train.fillna(0)
test_x = test_x.fillna(0)



In [147]:
!pip install --upgrade pytorch_tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [148]:
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabModel
from pytorch_tabnet.tab_model import TabNetClassifier


class TabNetClassifier(nn.Module):
    def __init__(self, num_features, num_classes):
        super(TabNetClassifier, self).__init__()
        
        self.tabnet = TabModel(
            input_dim= 1397,
            output_dim= 3,
            n_d=64,
            n_a=64,
            n_steps=6,
            gamma=1.5,
            lambda_sparse=0.001,
            momentum=0.3,
            clip_value=2,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=2e-2),
            mask_type="sparsemax",
            scheduler_params=dict(
                mode="min",
                patience=25,
                min_lr=1e-5,
                factor=0.5,
                verbose=True
            )
        )

    def forward(self, x):
      
        return self.tabnet(x)

In [149]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

model = TabNetClassifier(1397,3)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
# criterion = nn.CrossEntropyLoss()

# epochs = 10

# for epoch in range(epochs):
#     for x, y in train_loader:
#         optimizer.zero_grad()
#         output = model(x)
#         loss = criterion(output, y)
#         loss.backward()
#         optimizer.step()

#     with torch.no_grad():
#         valid_loss = 0
#         for x, y in valid_loader:
#             output = model(x)
#             loss = criterion(output, y)
#             valid_loss += loss.item()
#         valid_loss /= len(valid_loader)

#     print(f"Epoch {epoch+1} loss: {valid_loss}")
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(x_train[i])
    x_train[i] = le.transform(x_train[i])
    
    # for label in np.unique(x_test[i]): 
    #     if label not in le.classes_: 
    #         le.classes_ = np.append(le.classes_, label)
    # x_test[i] = le.transform(x_test[i])
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])  
print('Done.')

Done.


In [150]:
x_train = x_train.values
y_train = y_train.values
test_x = test_x.values

In [151]:

model.fit(x_train, y_train)
print('Done.')

/usr/local/lib/python3.8/dist-packages/pytorch_tabnet/abstract_model.py:665: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.0     |  0:00:00s
epoch 1  | loss: 0.0     |  0:00:00s
epoch 2  | loss: 0.0     |  0:00:00s
epoch 3  | loss: 0.0     |  0:00:00s
epoch 4  | loss: 0.0     |  0:00:00s
epoch 5  | loss: 0.0     |  0:00:00s
epoch 6  | loss: 0.0     |  0:00:00s
epoch 7  | loss: 0.0     |  0:00:00s
epoch 8  | loss: 0.0     |  0:00:00s
epoch 9  | loss: 0.0     |  0:00:00s
epoch 10 | loss: 0.0     |  0:00:00s
epoch 11 | loss: 0.0     |  0:00:00s
epoch 12 | loss: 0.0     |  0:00:00s
epoch 13 | loss: 0.0     |  0:00:00s
epoch 14 | loss: 0.0     |  0:00:00s
epoch 15 | loss: 0.0     |  0:00:00s
epoch 16 | loss: 0.0     |  0:00:00s
epoch 17 | loss: 0.0     |  0:00:00s
epoch 18 | loss: 0.0     |  0:00:00s
epoch 19 | loss: 0.0     |  0:00:00s
epoch 20 | loss: 0.0     |  0:00:00s
epoch 21 | loss: 0.0     |  0:00:00s
epoch 22 | loss: 0.0     |  0:00:00s
epoch 23 | loss: 0.0     |  0:00:00s
epoch 24 | loss: 0.0     |  0:00:00s
epoch 25 | loss: 0.0     |  0:00:00s
epoch 26 | loss: 0.0     |  0:00:00s
e

In [153]:
preds = model.predict(test_x)
print('Done.')

Done.


In [154]:
submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = preds
submit.to_csv('TabNetB.csv', index=False)